# 🏦 Customer Churn Analysis - EDA & Insights  

## 📌 Project Overview  
This notebook focuses on **Exploratory Data Analysis (EDA)** for our **European Banking Customer Churn dataset**. The goal is to uncover patterns, trends, and key factors influencing customer churn through visual and statistical analysis.  

## 📂 Dataset Information  
- **Dataset Name:** `european_customer_churn_data.parquet`  
- **Total Customers:** 1,008,530  
- **Key Features:**  
  - **Demographics:** Age, gender, geography, employment status  
  - **Financial Data:** Credit score, balance, salary, transactions  
  - **Behavioral Data:** Login frequency, mobile vs. branch transactions  
  - **Banking Issues:** Account problems, loan concerns, customer complaints  
  - **Sentiment & Small Talk:** Personalized interactions based on past experiences  

## 🔍 Notebook Structure  
### **1️⃣ Data Loading & Preprocessing**  
- Load `parquet` dataset into Pandas  
- Handle missing values, incorrect data entries, and standardize column formats  

### **2️⃣ Exploratory Data Analysis (EDA)**  
#### ✨ **Demographic Insights**  
- Age distribution and its relation to churn  
- Gender-wise and geography-based churn patterns  

#### 📊 **Financial Behavior Analysis**  
- Credit score vs. churn trends  
- Salary & balance distribution for active vs. churned customers  
- Transaction frequency and spending behavior  

#### 🔄 **Customer Interaction & Sentiment Analysis**  
- Common banking issues reported  
- Sentiment distribution across churned vs. non-churned customers  
- Impact of personalized small talk on churn rates  

#### 🏦 **High-Value Customers & Dedicated RMs**  
- Identification of VIP customers  
- Agent performance & escalation trends  

### **3️⃣ Feature Engineering for ML (Future Scope)**  
- Creating new features from transaction patterns  
- Encoding categorical variables for modeling  

### **4️⃣ Insights & Recommendations**  
- Key takeaways from data analysis  
- Business recommendations to reduce churn  

---

## 🎯 Goals & Expected Outcomes  
✔️ Identify **churn drivers** from customer demographics & banking issues  
✔️ Detect **high-risk customers** early for proactive retention strategies  
✔️ Provide **data-backed recommendations** for improving banking services  

---

**🛠️ Let’s dive into the analysis! 🚀**

In [1]:
!pip install pandas numpy matplotlib seaborn plotly

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/14.8 MB ? eta -:--:--
   - -------------------------------------- 0.5/14.8 MB 5.4 MB/s eta 0:00:03
   ---- ----------------------------------- 1.6/14.8 MB 6.0 MB/s eta 0:00:03
   -------- ------------------------------- 3.1/14.8 MB 6.6 MB/s eta 0:00:02
   ------------ --------------------------- 4.7/14.8 MB 6.9 MB/s eta 0:00:02
   ---------------- ----------------------- 6.3/14.8 MB 7.1 MB/s eta 0:00:02
   --------------------- ------------------ 7.9/14.8 MB 7.1 MB/s eta 0:00:01
   -------------------------- ------------- 9.7/14.8 MB 7.4 MB/s eta 0:00:01
   ------------------------------- -------- 11.8/14.8 MB 7.7 MB/s eta 0:00:01
   -------------------------------------- - 14.4/14.8 MB 8.3 MB/s eta 0:00:01
   ---------------------------------------- 14.8/14.8 MB 8.2 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install missingno

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
# Import core libraries
import pandas as pd
import numpy as np

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Missing values visualization
import missingno as msno

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer

In [4]:
# Display settings
plt.style.use("ggplot")
sns.set_theme(style="whitegrid")

In [5]:
# Ignore warnings for better readability
import warnings
warnings.filterwarnings("ignore")

In [9]:
# Load dataset
local_storage_path = "C:/Users/Nandan/GenAIProjects/ChurnData.parquet"
df = pd.read_parquet(local_storage_path )

# Display first few rows
df.head(10)

,CustomerID,Name,Age,Tenure,Gender,City,EmploymentStatus,CreditScore,Balance,EstimatedSalary,...,LoanAmount,OverdueLoan,LoginsLastMonth,TransactionsMobile,TransactionsBranch,Referrals,FamilyLinkedAccounts,MobileVsBranch,SupportTickets,CustomerComplaints
0,61059466,Misty Rhodes,69,27,Female,Copenhagen,Employed,758.0,232647.03,68055.33,...,21743.82,0,28,1,10,No,No,Mobile,2,0
1,84119748,Jamie Johnson,75,49,Mle,Berlin,Retired,422.0,68105.58,49612.55,...,21087.43,1,6,44,2,No,No,Branch,8,0
2,27358799,Stephen Rodriguez,49,48,Male,Stockholm,Student,629.0,151449.31,107608.61,...,35437.38,0,3,25,5,No,No,Mobile,5,0
3,70823244,Terri Joseph,78,12,Female,Paris,Employed,483.0,192763.97,53729.10,...,44502.37,0,27,49,13,No,No,Branch,8,0
4,87348217,Lauren Beard,35,16,Mle,Paris,Unemployed,NaN,214415.46,97974.80,...,35591.63,0,11,40,1,No,No,Mobile,9,0
5,43945592,Rhonda Harris,67,26,Femle,London,Self-employed,847.0,23792.45,83918.37,...,9801.23,1,2,40,7,No,No,Mobile,3,0
6,59327365,Joseph Taylor,24,20,Female,Berlin,Unemployed,356.0,200092.44,53356.30,...,31123.28,0,13,25,3,No,No,Mobile,5,0
7,76295275,Douglas Martin,30,2,Female,Stockholm,Unemployed,324.0,225325.78,41011.99,...,13082.36,0,17,39,15,No,No,Mobile,1,0
8,66358792,Mike Meza,38,19,Mle,Amsterdam,Self-employed,479.0,61398.02,36306.95,...,17582.38,0,28,29,7,No,No,Mobile,6,0
9,69790702,Matthew Cole,72,26,Mle,Copenhagen,Self-employed,422.0,104257.09,65920.65,...,14723.09,0,23,29,19,No,No,Mobile,3,0


In [10]:
# Basic info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1043828 entries, 0 to 1043827
Data columns (total 25 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   CustomerID            1043828 non-null  int64  
 1   Name                  1043828 non-null  object 
 2   Age                   1043828 non-null  int64  
 3   Tenure                1043828 non-null  int64  
 4   Gender                1043828 non-null  object 
 5   City                  1043828 non-null  object 
 6   EmploymentStatus      1043828 non-null  object 
 7   CreditScore           991491 non-null   float64
 8   Balance               970833 non-null   float64
 9   EstimatedSalary       1001915 non-null  float64
 10  Churn                 1043828 non-null  int64  
 11  MonthlyTransactions   1043828 non-null  int32  
 12  MonthlyDeposits       1043828 non-null  float64
 13  MonthlyWithdrawals    1043828 non-null  float64
 14  LoanType              834789 non-n

In [11]:
# Summary statistics
df.describe()

,CustomerID,Age,Tenure,CreditScore,Balance,EstimatedSalary,Churn,MonthlyTransactions,MonthlyDeposits,MonthlyWithdrawals,LoanAmount,OverdueLoan,LoginsLastMonth,TransactionsMobile,TransactionsBranch,SupportTickets,CustomerComplaints
count,1.043828e+06,1.043828e+06,1.043828e+06,991491.000000,970833.000000,1.001915e+06,1.043828e+06,1.043828e+06,1.043828e+06,1.043828e+06,834789.000000,1.043828e+06,1.043828e+06,1.043828e+06,1.043828e+06,1.043828e+06,1.043828e+06
mean,5.504092e+07,5.396053e+01,1.923653e+01,575.109488,124987.042923,1.099245e+05,1.195925e-01,2.450109e+01,9.995659e+03,9.998493e+03,25502.253241,8.021436e-02,1.501680e+01,2.499549e+01,9.991653e+00,4.501109e+00,5.027840e-02
std,2.597850e+07,2.105741e+01,1.416313e+01,158.883290,72140.002541,5.199623e+04,3.244846e-01,1.442361e+01,5.770942e+03,5.772967e+03,14135.548199,2.716249e-01,8.940607e+00,1.471909e+01,6.051749e+00,2.872293e+00,2.185189e-01
min,1.000007e+07,1.800000e+01,0.000000e+00,300.000000,0.340000,2.000013e+04,0.000000e+00,0.000000e+00,7.549152e-03,1.991612e-02,1000.050000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.253750e+07,3.600000e+01,8.000000e+00,438.000000,62471.800000,6.480075e+04,0.000000e+00,1.200000e+01,5.003381e+03,5.002470e+03,13266.110000,0.000000e+00,7.000000e+00,1.200000e+01,5.000000e+00,2.000000e+00,0.000000e+00
50%,5.503627e+07,5.400000e+01,1.600000e+01,575.000000,124947.160000,1.098903e+05,0.000000e+00,2.400000e+01,9.990176e+03,9.995976e+03,25510.020000,0.000000e+00,1.500000e+01,2.500000e+01,1.000000e+01,4.000000e+00,0.000000e+00
75%,7.754337e+07,7.200000e+01,3.000000e+01,713.000000,187436.610000,1.548461e+05,0.000000e+00,3.700000e+01,1.499289e+04,1.499341e+04,37730.200000,0.000000e+00,2.300000e+01,3.800000e+01,1.500000e+01,7.000000e+00,0.000000e+00
max,9.999997e+07,9.000000e+01,5.000000e+01,850.000000,249999.910000,2.000000e+05,1.000000e+00,4.900000e+01,1.999999e+04,1.999996e+04,49999.900000,1.000000e+00,3.000000e+01,5.000000e+01,2.000000e+01,9.000000e+00,1.000000e+00


In [12]:
# Check missing values
missing_values = df.isnull().sum()
missing_values[missing_values > 0]

CreditScore         52337
Balance             72995
EstimatedSalary     41913
LoanType           209039
LoanAmount         209039
dtype: int64

In [13]:
df.columns

Index(['CustomerID', 'Name', 'Age', 'Tenure', 'Gender', 'City',
       'EmploymentStatus', 'CreditScore', 'Balance', 'EstimatedSalary',
       'Churn', 'MonthlyTransactions', 'MonthlyDeposits', 'MonthlyWithdrawals',
       'LoanType', 'LoanAmount', 'OverdueLoan', 'LoginsLastMonth',
       'TransactionsMobile', 'TransactionsBranch', 'Referrals',
       'FamilyLinkedAccounts', 'MobileVsBranch', 'SupportTickets',
       'CustomerComplaints'],
      dtype='object')

In [15]:
df['Gender'].unique()

array(['Female', 'Mle', 'Male', 'Femle'], dtype=object)

In [16]:
# Replace incorrect gender entries with correct ones
df['Gender'] = df['Gender'].replace({'Mle': 'Male', 'Femle': 'Female'})

# Verify the changes
print(df['Gender'].unique())

['Female' 'Male']


In [ ]:
df['EmploymentStatus'].unique()

array(['Employed', 'Retired', 'Student', 'Unemployed', 'Self-employed'],
      dtype=object)

In [18]:
df['LoanType'].unique()

array(['Auto', 'Student', 'Mortgage', 'Personal', None], dtype=object)

In [19]:
df["LoanType"].fillna("No Loan", inplace=True)
df["LoanAmount"].fillna(0, inplace=True)

In [20]:
df["Balance"].fillna(0, inplace=True)

In [22]:
# Identify features to use to impute the CreditScore with appropriate Calculation
features = ["Balance", "EstimatedSalary", "Age", "Tenure", "LoanAmount"]
df_train = df[df["CreditScore"].notnull()]
df_pred = df[df["CreditScore"].isnull()]

In [ ]:
# Prepare training data
X_train = df_train[features]
y_train = df_train["CreditScore"]

In [24]:
# Handle missing values in features
imputer = SimpleImputer(strategy="median")
X_train = imputer.fit_transform(X_train)

In [26]:
# Train the model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

MemoryError: could not allocate 134217728 bytes

In [ ]:
# Predict missing CreditScore values
X_pred = imputer.transform(df_pred[features])
df.loc[df["CreditScore"].isnull(), "CreditScore"] = model.predict(X_pred)

In [ ]:
# Check if all missing values are filled
print(df["CreditScore"].isnull().sum())